In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY="5g"
spark = SparkSession.builder.appName("taxi_fare_pred")\
        .config("spark.executor.memory", MAX_MEMORY)\
        .config("spark.driver.memory", MAX_MEMORY)\
        .getOrCreate()

In [3]:
data_dir = "/Users/ji/data-engineering/01-spark/data/"

In [4]:
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")

In [8]:
toy_df = train_df.sample(False, 0.1, seed=1)

In [9]:
train_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [10]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

stages = []

for c in cat_feats:
    cat_indexer = StringIndexer(inputCol=c, outputCol=c + "_idx").setHandleInvalid('keep')
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c+"_onehot"])
    stages += [cat_indexer, onehot_encoder]

In [11]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

num_feats = [
    "passenger_count",
    "trip_distance",
    "pickup_time"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol = n+"_vector")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=n+"_scaled")
    stages += [num_assembler, num_scaler]

In [12]:
assembler_inputs = [c+"_onehot" for c in cat_feats] + [n+"_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol='feature_vector')
stages += [assembler]

## Hyperparameter Tuning

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

lr = LinearRegression(
    maxIter=30,
    solver="normal",
    labelCol="total_amount",
    featuresCol="feature_vector"
)

cv_stages = stages + [lr]

In [15]:
cv_pipeline = Pipeline(stages=cv_stages)

In [16]:
param_grid = ParamGridBuilder()\
                .addGrid(lr.elasticNetParam, [0.1,0.2,0.3,0.4,0.5])\
                .addGrid(lr.regParam, [0.01,0.02,0.03,0.04,0.05])\
                .build()

In [17]:
cross_val = CrossValidator(estimator=cv_pipeline,
                          estimatorParamMaps=param_grid,
                          evaluator=RegressionEvaluator(labelCol="total_amount"),
                          numFolds=5)

In [18]:
cv_model = cross_val.fit(toy_df)

22/06/14 00:35:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/06/14 00:35:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/06/14 00:35:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/06/14 00:35:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [24]:
alpha = cv_model.bestModel.stages[-1]._java_obj.getElasticNetParam()
reg_param = cv_model.bestModel.stages[-1]._java_obj.getRegParam()

## Train

In [25]:
transform_stages = stages
pipeline = Pipeline(stages=transform_stages)
fitted_transformer = pipeline.fit(train_df)

In [26]:
vtrain_df = fitted_transformer.transform(train_df)

In [27]:
lr = LinearRegression(
    maxIter=5,
    solver = "normal",
    labelCol="total_amount",
    featuresCol = "feature_vector",
    elasticNetParam=alpha,
    regParam=reg_param
)

In [28]:
vtrain_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vector: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nullable =

In [29]:
model = lr.fit(vtrain_df)

In [30]:
vtest_df = fitted_transformer.transform(test_df)

In [31]:
pred = model.transform(vtest_df)

In [32]:
pred.cache()

DataFrame[passenger_count: double, pickup_location_id: bigint, dropoff_location_id: bigint, trip_distance: double, pickup_time: int, day_of_week: string, total_amount: double, pickup_location_id_idx: double, pickup_location_id_onehot: vector, dropoff_location_id_idx: double, dropoff_location_id_onehot: vector, day_of_week_idx: double, day_of_week_onehot: vector, passenger_count_vector: vector, passenger_count_scaled: vector, trip_distance_vector: vector, trip_distance_scaled: vector, pickup_time_vector: vector, pickup_time_scaled: vector, feature_vector: vector, prediction: double]

In [33]:
pred.select(['trip_distance', 'day_of_week', 'total_amount', 'prediction']).show()

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|         13.9|   Saturday|         4.3| 128.5649686193176|
|          3.3|     Sunday|         5.8|18.306511368188804|
|          5.5|     Sunday|       26.75| 22.91940540741578|
|          7.9|   Saturday|        30.3|  29.0141363365863|
|          0.4|  Wednesday|         7.8|12.407297064995502|
|          1.0|   Saturday|        11.8|12.989476879131605|
|          1.3|   Saturday|       12.85|14.043332199411283|
|          9.4|   Saturday|        39.8| 34.41571386022454|
|          1.5|  Wednesday|        13.3|15.342927953668063|
|          4.7|     Sunday|       26.15| 21.34015642997333|
|          0.6|   Saturday|       11.75|13.326841158864733|
|          1.1|    Tuesday|        11.8| 14.50709892088285|
|          3.7|     Friday|       24.35|19.542350783223654|
|          2.4|   Saturday|       14.75|

In [34]:
model.summary.rootMeanSquaredError

5.703485487910037

In [35]:
model.summary.r2

0.8042133371871026

In [36]:
model_dir = "/Users/ji/data-engineering/01-spark/data/model"
model.save(model_dir)

In [37]:
from pyspark.ml.regression import LinearRegressionModel

In [38]:
lr_model = LinearRegressionModel().load(model_dir)

In [39]:
preds = lr_model.transform(vtest_df)

In [40]:
preds.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|pickup_location_id_idx|pickup_location_id_onehot|dropoff_location_id_idx|dropoff_location_id_onehot|day_of_week_idx|day_of_week_onehot|passenger_count_vector|passenger_count_scaled|trip_distance_vector|trip_distance_scaled|pickup_time_vector|  pickup_time_scaled|      feature_vector|        prediction|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+----------------------